## Importing Libraries

In [1]:
import pandas as pd
films = pd.read_csv('films_clean.csv')

In [2]:
films.rename(columns= {'id':'film_id', 'title': 'film_title'}, inplace=True)

## Fixing the genres column for my SQL database

In [3]:
#I created a genre ID in order to do my EDA
films_ = films.copy()
films_['genres'] = films_['genres'].str.split(',')

In [4]:
films_exploded = films_.explode('genres')

In [5]:
films_['genres'] = films_['genres'].str.split(',')

In [6]:
genres_table = pd.DataFrame(columns = ['genre_id','genre'])
for i, genre in enumerate(set(films_exploded['genres'].str.strip())):
    genres_table.loc[i] = {'genre_id':i, 'genre':genre}

genres_table.drop_duplicates(inplace=True)

In [7]:
film_genre_table = films_exploded
films_exploded['genres'] = films_exploded['genres'].str.strip()
film_genre_table = film_genre_table.merge(genres_table, left_on='genres', right_on='genre')
film_genre_table = film_genre_table[['film_id','genre_id', 'director_gender']]

## Fixing the countries column for my SQL database

In [8]:
films_['production_countries'] = films_['production_countries'].str.split(',')

In [9]:
films_exploded_countries = films_.explode('production_countries')

In [10]:
countries_table = pd.DataFrame(columns = ['country_id','country'])
for i, country in enumerate(set(films_exploded_countries['production_countries'].str.strip())):
    countries_table.loc[i] = {'country_id':i, 'country':country}

In [11]:
film_country_table = films_exploded_countries
films_exploded_countries['production_countries'] = films_exploded_countries['production_countries'].str.strip()
film_country_table = film_country_table.merge(countries_table, left_on='production_countries', right_on='country')
film_country_table = film_country_table[['film_id','country_id', 'director_gender']]

## Creating new IDs for my SQL database

## Directors

In [12]:
films['director_id'] = pd.factorize(films['director'])[0]+1

## Ratings

In [13]:
films['rating_id'] = pd.factorize(films['popularity'])[0]+1

## Creating new dataframes

## Films

In [14]:
#I am making a copy everytime to make sure I don't edit my original dataframe
films_table = films[['film_id', 'film_title','original_language', 'overview', 'year', 'popularity', 'director_id', "spoken_languages"]].copy()
films_table.drop_duplicates(inplace=True)


## Ratings

In [15]:
ratings = films[['film_id', 'popularity', 'rating_id']].copy()
ratings.drop_duplicates(inplace=True)

## Directors

In [16]:
directors = films[['director_id', 'director_gender', 'director', 'film_id']].copy()
directors.drop_duplicates(inplace=True)

## Exporting to SQL

In [17]:

import pymysql.cursors
from sqlalchemy import create_engine
from sqlalchemy import text


import getpass
sql_pass = getpass.getpass()


connection_string = 'mysql+pymysql://root:' + sql_pass + '@localhost:3306/film_data'
engine = create_engine(connection_string)

In [18]:


genres_table.to_sql('genres',engine, 'film_data', if_exists='replace', index=False)
countries_table.to_sql('countries', engine,'film_data', if_exists='replace', index=False)
directors.to_sql('directors',engine, 'film_data', if_exists='replace', index=False)
film_genre_table.to_sql('film_genre',engine, 'film_data', if_exists='replace', index=False)
films_table.to_sql('films',engine, 'film_data', if_exists='replace', index=False)
film_country_table.to_sql('film_country',engine, 'film_data', if_exists='replace', index=False)




## Exporting to CSVs 

In [19]:
films.to_csv('films_clean.csv', index=False)

In [20]:
film_genre_table.to_csv('film_genres.csv', index=False)

In [21]:
genres_table.to_csv('genres_names.csv', index=False)

In [22]:
film_country_table.to_csv('film_country.csv', index=False)

In [23]:
countries_table.to_csv('countries_names.csv', index=False)